**This file is primarily for seeing how the model would be trained, it wasn't run and was used later on to build up our proper training**

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
pip install kagglehub[pandas-datasets]

In [13]:
# Install the Kaggle API
!pip install kaggle

# Set up Kaggle credentials
# You'll need to upload your kaggle.json file to Colab
from google.colab import files
files.upload()  # Upload your kaggle.json file

# Move the kaggle.json file to the correct location
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d getcam/cod10k
!unzip cod10k.zip

# Now let's convert the data to a dataframe
import pandas as pd
import os
from PIL import Image
import numpy as np

# The structure of COD10K will depend on the actual data format
# Let's assume there's an image directory and possibly annotations

# Example: Create a dataframe with image paths and basic information
image_dir = "COD10K/Train/Image"  # Adjust this path based on the actual dataset structure
image_files = []
image_sizes = []

# Collect image info
for root, _, files in os.walk(image_dir):
    for file in files:
        if file.endswith(('.jpg', '.png', '.jpeg')):
            img_path = os.path.join(root, file)
            image_files.append(img_path)

            # Get image dimensions
            with Image.open(img_path) as img:
                width, height = img.size
                image_sizes.append((width, height))

# Create dataframe
df = pd.DataFrame({
    'image_path': image_files,
    'width': [size[0] for size in image_sizes],
    'height': [size[1] for size in image_sizes],
    'filename': [os.path.basename(path) for path in image_files]
})

print(df.head())

# the way we're making the model better is through retraining the current model by
# using the resume part of the model and then adding the cod dataset but with the
# bounding boxes and metadata made for the new dataset that is edited and
# made according to the required yaml format, for classification, we might be able to use
# the lila dataset but ill be using the already given pytorchwildlife or the kaggle
# MD updated model with species classifier.
# i did this on my Laptop while all my datasets are saved on my PC, downloading the dataset
# will take 8 hours (for 2gb) and generally just setting up the environment to download
# the imports take an hour atleast due to LUMS internet

Streaming output truncated to the last 5000 lines.
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-25-Centipede-1656.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-25-Centipede-1657.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-25-Centipede-1660.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-25-Centipede-1661.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-25-Centipede-1662.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-26-Chameleon-1664.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-26-Chameleon-1665.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-26-Chameleon-1666.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-26-Chameleon-1668.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-26-Chameleon-1669.jpg  
  inflating: COD10K-v3/Train/Image/COD10K-CAM-2-Terrestrial-26-Chameleon-1672.jpg  
  inflating: COD10K-v3/Tr

In [ ]:
# import os
# !git clone https://github.com/ultralytics/yolov5
# os.chdir("/kaggle/working/yolov5/")
# !pip install -r requirements.txt

# !curl --keepalive -OL https://github.com/microsoft/CameraTraps/releases/download/v5.0/md_v5a.0.0.pt

# !cp -r /kaggle/input/megadetector-tutorial/dataset/ /kaggle/working/

# !python train.py --weights /kaggle/working/yolov5/md_v5a.0.0.pt --epochs 10 --batch-size 14 --device 0 --data /kaggle/working/dataset/dataset.yaml --imgsz 1280 --freeze 12

# !python detect.py --name test --device 0 --weights /kaggle/working/yolov5/runs/train/exp/weights/best.pt --imgsz 1280 --exist-ok --conf-thres 0.1 --source /kaggle/working/dataset/test/images/

# from IPython.display import Image
# Image("runs/detect/test/magpie_0143.JPG")

# Image("runs/detect/test/raven_0013.JPG")

# Image("runs/detect/test/Image_0249.JPG")



In [9]:
import json
from pathlib import Path

def coco_to_yolo(coco_json_path, output_dir):
    with open(coco_json_path) as f:
        data = json.load(f)

    # Create category map (0=animal, 1=non-animal)
    category_map = {cat['id']: 0 if 'animal' in cat['name'].lower() else 1
                   for cat in data['categories']}

    for img in data['images']:
        img_id = img['id']
        anns = [a for a in data['annotations'] if a['image_id'] == img_id]

        yolo_lines = []
        for ann in anns:
            cat_id = category_map[ann['category_id']]
            x, y, w, h = ann['bbox']
            # Convert to normalized center coordinates
            x_center = (x + w/2) / img['width']
            y_center = (y + h/2) / img['height']
            nw = w/img['width']
            nh = h/img['height']

            yolo_lines.append(f"{cat_id} {x_center} {y_center} {nw} {nh}")

        # Write to label file
        label_path = Path(output_dir) / f"{Path(img['file_name']).stem}.txt"
        with open(label_path, 'w') as f:
            f.write("\n".join(yolo_lines))

drive  sample_data


In [ ]:
model_name: MDV6-yolov10-e
imgsz: 1280
epochs: 100
batch_size_train: 8
lr0: 0.001

augmentations:
  - name: MotionBlur
    probability: 0.3
  - name: RandomShadow
    probability: 0.4
  - name: ColorJitter
    brightness: 0.2
    contrast: 0.2
    saturation: 0.2
    hue: 0.1

In [ ]:
# python main.py --freeze-backbone --epochs 50

In [ ]:
python main.py --unfreeze-all --epochs 50 --lr0 0.0001

In [ ]:
def evaluate_camouflage_performance(test_dir):
    # Custom metrics for ecological relevance
    from pycocotools.coco import COCO
    from pycocotools.cocoeval import COCOeval

    coco_gt = COCO("path/to/coco_annotations.json")
    coco_dt = coco_gt.loadRes("path/to/predictions.json")

    # Focus on biologically relevant metrics
    coco_eval = COCOeval(coco_gt, coco_dt, 'bbox')
    coco_eval.params.iouThrs = [0.25]  # Lower IoU threshold for partial visibility
    coco_eval.evaluate()
    coco_eval.accumulate()
    coco_eval.summarize()

    # Camouflage-Specific Scores
    print(f"Camouflaged mAP@0.25: {coco_eval.stats[0]:.3f}")
    print(f"Hard Case Recall: {coco_eval.stats[8]:.3f}")  # AR@100

In [ ]:
class CamouflagePostProcessor:
    def __init__(self, model):
        self.model = model
        self.ecological_classes = ['aquatic', 'terrestrial', 'aerial']

    def predict(self, image):
        preds = self.model(image)

        # Ecological context filtering
        filtered = []
        for det in preds:
            if self.is_camouflage_context(det):
                filtered.append(det)
        return filtered

    def is_camouflage_context(self, detection):
        # Implement biological rules
        if detection.class == 'aquatic' and detection.confidence < 0.3:
            return False  # Filter false water reflections
        # Add species-specific rules
        return True

In [ ]:
def field_validation(model, camera_trap_images):
    results = []
    for img_path in camera_trap_images:
        preds = model.predict(img_path)
        annotated = visualize_predictions(img_path, preds)

        # Expert validation
        expert_score = get_biologist_rating(annotated)
        results.append({
            'image': img_path,
            'machine_detections': len(preds),
            'expert_score': expert_score
        })
    return results

In [ ]:
python main.py --resume --weights best.pt \
--data new_field_data.yaml \
--epochs 20 --lr0 0.00001